**Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto**

## Installing relevant packages

In [1]:
!pip install geopy

In [49]:
!pip install geocoder 

     |████████████████████████████████| 102kB 2.4MB/s 


In [50]:
!pip install folium

## Importing libraries

In [51]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library
import geocoder # import geocoder
print('Libraries imported.')

Libraries imported.


In [0]:
df = pd.read_csv('toronto.csv')

## Reading dataset file and manipulating the DataFrame 

In [53]:
df.iloc[6,2]='Queen\'s Park'
df[df['Borough']=='Queen\'s Park']

,Postcode,Borough,Neighbourhood
6,M9A,Queen's Park,Queen's Park


In [54]:
df1 = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x:','.join(x.astype(str))).reset_index()
df1[df1['Postcode']=="M4G"]

,Postcode,Borough,Neighbourhood
38,M4G,East York,Leaside


Shape of the final cleaned dataset

In [55]:
df1.shape

(103, 3)

In [72]:
df3 = pd.read_csv('Geospatial_Coordinates.csv')
df3.head()
df3.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [0]:
lat = []
lon = []
for i in range(0,df3.shape[0]):
  lat.append(df3.iloc[i,1])
  lon.append(df3.iloc[i,2])
df1['Latiude']=lat
df1['Longitude']=lon

In [88]:
df1.head()

,Postcode,Borough,Neighbourhood,Latiude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
